In [1]:
line_data = 'C:/Users/jackp/Research/spatical_scan/file.txt'
repo_name = 'C:/Users/jackp/Research/repository/android-job'
until_data = '2019-6-18'
since_data = '2018-6-11'
dot_name = 'C:/Users/jackp/Research/spatical_scan/all_jar.dot'

In [2]:
#変更されたノードの取得
from git import *
import datetime, time


def change_node(repo_name, until_data, since_data, branch_name):

    #コミットリストの作成
    list1 = []
    repo = Repo(repo_name)
    #print('hexsha,author,authored_date,file_name,deletions,lines,insertions')
    for item in repo.iter_commits(branch_name , until = until_data ,since = since_data):
        file_list = item.stats.files
        for file_name in file_list:
        #    dt = datetime.datetime.fromtimestamp(item.authored_date).strftime("%Y-%m-%d %H:%M:%S")
        #    insertions = file_list.get(file_name).get('insertions')
        #    deletions = file_list.get(file_name).get('deletions')
        #    lines = file_list.get(file_name).get('lines')
             list1.append(file_name)

    list1 =list(set(list1))
    
    return list1

In [3]:
node_list = change_node(repo_name, until_data, since_data, branch_name = 'master')

In [4]:
node_list =list(set(node_list))

In [5]:
#テキストデータ読み込み
col0 = []
col1 = []
for i, line in enumerate(open(line_data,'r',encoding="utf-8")): # ファイルを開いて一行一行読み込む
    c = line.lstrip().split(" ")
    c[0]=c[0].strip()
    c[1]=c[1].strip()
    col0.append((c[0]))
    col1.append((c[1]))


#ファイル行数のリストを作成
file_lines = []
for x in col1:
    file_lines.append(x)        # x : ファイル名   y : ファイル行数

In [17]:
#txtファイル→グラフへの変換
col0 = [] #始点の頂点集合
col1 = [] #終点の頂点集合
for i, line in enumerate(open(dot_name)): # ファイルを開いて一行一行読み込む
    
    c = line.split("->")
    c[0]=c[0].strip()
    c[0]=c[0].strip('  ')
    c[0]=c[0].strip('\"')
    c[1]=c[1].strip()
    c[1]=c[1].strip('  ')
    c[1]=c[1].strip(";")
    c[1]=c[1].strip('\"')
    
    c[0] =  c[0].replace('.', '/')
    #c[0] =  'src/' +  c[0] + '.java'
    c[0] =  c[0] + '.java'
    col0.append((c[0]))
    
    c[1] =  c[1].replace('.', '/')
    #c[1] =  'src/' +  c[1] + '.java'
    c[1] =  c[1] + '.java'
    col1.append((c[1]))
    
#辺のリストを作成
java_module = []
for x, y in zip(col0, col1):
    java_module.append([x,y])
    
#java_moduleはモジュール依存関係のグラフ
java_module

[['android/support/v4/app/SafeJobIntentService.java', 'android/os/Build.java'],
 ['android/support/v4/app/SafeJobIntentService.java',
  'android/os/Build$VERSION.java'],
 ['android/support/v4/app/SafeJobIntentService.java',
  'android/support/annotation/RestrictTo.java'],
 ['android/support/v4/app/SafeJobIntentService.java',
  'android/support/annotation/RestrictTo$Scope.java'],
 ['android/support/v4/app/SafeJobIntentService.java',
  'android/support/v4/app/JobIntentService.java'],
 ['android/support/v4/app/SafeJobIntentService.java',
  'android/support/v4/app/JobIntentService$CompatJobEngine.java'],
 ['android/support/v4/app/SafeJobIntentService.java',
  'android/support/v4/app/JobIntentService$GenericWorkItem.java'],
 ['android/support/v4/app/SafeJobIntentService.java',
  'android/support/v4/app/SafeJobServiceEngineImpl.java'],
 ['android/support/v4/app/SafeJobIntentService.java',
  'java/lang/SecurityException.java'],
 ['android/support/v4/app/SafeJobServiceEngineImpl.java',
  'andr

In [40]:
#名前の書き換え
for i in file_lines:
    for j in java_module:
        if i.endswith(j[0]): #文字列の一致を見る.
            j[0] = i
        elif i.endswith(j[1]): #文字列の一致を見る.
            j[1] = i
            
#該当の依存関係の抽出
#(java_commitにあるノードに関係あるやつを見つける)

dependency = []
for i in node_list:
    for j in java_module:
        if i in j:
            dependency.append(j)

#重複排除
dependency2 = []
for i in dependency:
    if i not in dependency2:
        dependency2.append(i)
        
#変更されたノード & 変更されたノードと隣接するノード　のリスト作成
node_list_L = []
for i in dependency2:
    if i[0] not in node_list_L:
        node_list_L.append(i[0])
    elif i[1] not in node_list_L:
        node_list_L.append(i[1])

In [45]:
#ノードクラスの定義
class Node:
    def __init__(self, node_name):
        self.name = node_name
        
    def node_name(self):
        return self.name

#エッジクラスの定義
class Edge:
    def __init__(self, start, end):
        self.start = start
        self.end = end
    def edge_elem(self):
        return [self.start, self.end]

In [54]:
#ノードクラスのリスト作成
def Node_create(node_list):
    
    node_obj_list = [0] * len(node_list)

    for i in range(len(node_obj_list)):
        node_obj_list[i] = Node(node_list[i])
    
    return node_obj_list

#エッジリストのリスト作成
def Edge_create(edge_list):
    
    edge_obj_list = [0] * len(edge_list)

    for i in range(len(edge_list)):
        edge_obj_list[i] = Edge(edge_list[i][0], edge_list[i][1])
    
    return edge_obj_list

In [58]:
a = Node_create(node_list_L)
b = Edge_create(dependency2)

In [59]:
#pandasデータフレームへデータを変換
import pandas as pd
modify_node = pd.DataFrame(node_list)
Edge_list   = pd.DataFrame(dependency2)
Node_L    =  pd.DataFrame(node_list_L)

In [61]:
#エクセルデータの名前を決定
Excel_name = '__evernote__'    #ここに名前を入力

modify_excel = 'ModifyNode' + Excel_name + '.xlsx'
Edge_excel   = 'Edge_list'   + Excel_name + '.xlsx'
Node_L_excel = 'Node_L' + Excel_name + '.xlsx'

In [62]:
#エクセルデータとして結果を保存
modify_node.to_excel(modify_excel, sheet_name='new_sheet_name')
Edge_list.to_excel(Edge_excel, sheet_name='new_sheet_name')
Node_L.to_excel(Node_L_excel, sheet_name='new_sheet_name')